In [1]:
# when not using colab
import sys
sys.path.append('../src/QA')
import reload_recursive
%reload word_embed
from word_embed import WordEmbed

In [17]:
%reload data_prep
from data_prep import DataPrep

All these functions should be moved to their own module, im just doing it like this as a rough draft.

In [5]:
import torch
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [19]:
df = DataPrep.parse_tsv('../WikiQA-train.tsv')
question_doc_raw_train = DataPrep.convert_pd_to_json(df)

In [20]:
%reload feat_extract
from feat_extract import FeatExt

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/max/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/max/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/max/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/max/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/max/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/max/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/max/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] D

In [21]:
%reload embed_doc
from embed_doc import EmbedAndConcat

In [22]:
train_q_inputs, train_doc_inputs, train_doc_targets = EmbedAndConcat.get_unrolled_embeddings(question_doc_raw_train)

In [23]:
df_test = DataPrep.parse_tsv('../WikiQA-test.tsv')
question_doc_raw_test = DataPrep.convert_pd_to_json(df_test)
test_q_inputs, test_doc_inputs, test_doc_targets = EmbedAndConcat.get_unrolled_embeddings(question_doc_raw_test)

In [24]:
%reload pad
from pad import Pad

In [25]:
q_max_len = max(Pad.get_max(train_q_inputs), Pad.get_max(test_q_inputs))
doc_max_len = max(Pad.get_max(train_doc_inputs), Pad.get_max(test_doc_inputs))

doc_len = doc_max_len
Pad.cut_pad_to(q_max_len, train_q_inputs)
Pad.cut_pad_to(q_max_len, test_q_inputs)
Pad.cut_pad_to(doc_len, train_doc_inputs)
Pad.cut_pad_to(doc_len, test_doc_inputs)
Pad.cut_pad_to(doc_len, train_doc_targets, target=True)
Pad.cut_pad_to(doc_len, test_doc_targets, target=True)
print(f"After padding train question min len {Pad.get_min(train_q_inputs)}")
print(f"After padding test doc min len {Pad.get_min(test_doc_inputs)}")

After padding train question min len 27
After padding test doc min len 1889


In [26]:
# convert the target tags into integers
# only OOA and IOA for now
target2int = {"OOA": 0, "IOA": 1, "BOA": 1, "EOA": 1}
int2target = {0: "OOA", 1: "IOA", 2: "BOA", 3: "EOA"}

Pad.convert_targets(train_doc_targets, target2int)
Pad.convert_targets(test_doc_targets, target2int)

In [27]:
%reload stat_helper
from stat_helper import StatHelper

In [28]:
training_class_weights = StatHelper.get_class_weights(train_doc_targets, 2)
print(training_class_weights)

[1.0, 122.84679467327346]


In [29]:
from torch.utils.data import TensorDataset
import torch
train_dataset = TensorDataset(torch.Tensor(train_q_inputs), torch.Tensor(train_doc_inputs), torch.LongTensor(train_doc_targets))

from torch.utils.data import DataLoader
batch_size = 128
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) 

test_question_tensor = torch.Tensor(test_q_inputs)
test_doc_tensor = torch.Tensor(test_doc_inputs)
test_target_tensor = torch.LongTensor(test_doc_targets)

/tmp/ipykernel_35181/3378231814.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525493953/work/torch/csrc/utils/tensor_new.cpp:230.)
  train_dataset = TensorDataset(torch.Tensor(train_q_inputs), torch.Tensor(train_doc_inputs), torch.LongTensor(train_doc_targets))


In [30]:
del train_q_inputs
del test_q_inputs
del train_doc_inputs
del test_doc_inputs
del train_doc_targets
del test_doc_targets

In [31]:
import gc
gc.collect()

7537

In [32]:
def train(input_question_tensor, input_document_tensor, target_tensor, question_model, document_model, question_optimizer, document_optimizer, criterion):
    loss = 0    

    question_optimizer.zero_grad()
    document_optimizer.zero_grad()
    # get output from question model
    question_output = question_model(input_question_tensor)

    # predictions from the document model with output from question for attention
    document_output = document_model(input_document_tensor, question_output)
    
    loss = criterion(document_output.view(-1, document_output.shape[-1]), target_tensor.view(-1))
    loss.backward()
    
    question_optimizer.step()
    document_optimizer.step()

    return loss.item()

In [33]:
import time
import math

# Helper functions for training
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [34]:
import random
def trainIters(question_model, document_model, n_iters, class_balance=None, print_every=5, plot_every=10, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    question_model_optimizer = optim.SGD(question_model.parameters(), lr=learning_rate)
    document_model_optimizer = optim.SGD(document_model.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss(weight=class_balance)
    
    for iter in range(1, n_iters):
        for question_input,doc_input,targets in train_loader:
            question_input = question_input.to(device)
            doc_input = doc_input.to(device)
            targets = targets.to(device)

            loss = train(question_input, doc_input, targets, question_model, document_model, question_model_optimizer, document_model_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

In [46]:
%reload model.QA_RNN
from model import QA_RNN
q_embed_size = list(train_loader)[0][0].shape[2]
doc_embed_size = list(train_loader)[0][1].shape[2]
hidden_size = 100
doc_rnn_model = QA_RNN.DocumentModel(doc_embed_size, hidden_size, 2).to(device)
question_rnn_model = QA_RNN.QuestionModel(q_embed_size, hidden_size).to(device)

In [47]:
trainIters(question_rnn_model, doc_rnn_model, 30, torch.Tensor(training_class_weights))
#trainIters(question_rnn_model, doc_rnn_model, 30)

1m 3s (- 5m 18s) (5 16%) 6.8757
2m 7s (- 4m 14s) (10 33%) 4.4627
3m 10s (- 3m 10s) (15 50%) 3.8020
4m 14s (- 2m 7s) (20 66%) 3.5908
5m 17s (- 1m 3s) (25 83%) 3.4949


In [48]:
from sklearn.metrics import classification_report
def evaluate(input_question_tensor, input_document_tensor, target_tensor, question_model, document_model):
    with torch.no_grad():
        loss = 0
        
        question_output = question_model(input_question_tensor)
        
        document_output = document_model(input_document_tensor, question_output)
        predicts = torch.argmax(document_output.view(-1, document_output.shape[-1]), dim=-1)
        print(classification_report(predicts, target_tensor.view(-1)))

In [49]:
evaluate(test_question_tensor, test_doc_tensor, test_target_tensor, question_rnn_model, doc_rnn_model)

tensor(1030667)
              precision    recall  f1-score   support

           0       0.87      1.00      0.93   1030667
           1       1.00      0.05      0.10    159403

    accuracy                           0.87   1190070
   macro avg       0.94      0.53      0.52   1190070
weighted avg       0.89      0.87      0.82   1190070

